In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from typing import Optional, Union
from pydantic import Field
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.messages import BaseMessage, SystemMessage, MessageLikeRepresentation

descriptions = {
    "status": """
        This field will be one of the following values:
        - Incomplete or erroneous
            Meaning: Code is incomplete or it is producing errors (it cannot be run succesfully).
        - Incorrect but better
            Meaning: Code is incorrect but better than previous code snippet in terms of syntax, logic and correctness. This means the user is moving in right direction towards the correct code.
        - Incorrect and worse
            Meaning: Code is incorrect and worse than previous code snippet in terms of syntax, logic -  This means the user is moving in wrong direction away from the correct code.
        - Incorrect and same as previous code snippet
            Meaning: Code is incorrect and same as previous code snippet and there is no improvement in the code.
        - Partially correct
            Meaning: Code is partially correct. It covers some of the requirements from the question but not all.
        - Correct but can be improved
            Meaning: Syntax is correct, code does not produce any error on running it and output is as expected BUT the code can be improved (improvements can be in terms of code optimization, better logic, better variable names, better comments, etc.)
        - Correct
            Meaning: Syntax is correct, code does not produce any error on running it and output is as expected. Code is following the guidelines provided in the question. There is no redundancy in the code. Code is optimized and well written.
        """,

    "score": """
        An approximate score out of 100 as the quality of the code snippet provided by user. This score should be based on the quality of code snippet and how well it is following the guidelines provided in the question.
    """,

    "result": """
        - Improving
            Meaning: The given code is better than the previous code snippet in terms of syntax, logic and correctness. This means the user is moving in right direction towards the correct code.
        - Not Improving
            Meaning: The given code is worse than the previous code snippet in terms of syntax, logic or correctness. This means the user is moving in wrong direction away from the correct code.
    """,

    "comment": """
        One line comment showing the reason for the given score.
    """
}


class Evaluation(BaseMessage):
    """Evaluation of the code provided by the user."""

    status: str = Field(default=None, description=descriptions["status"])
    score: str = Field(default=None, description=descriptions["score"])
    result: int = Field(default=0, description=descriptions["result"])
    comment: Optional[str] = Field(default=None, description=descriptions["comment"])

class ConversationalResponse(BaseMessage):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description= "A conversational response to the user's query")


class FinalResponse(BaseMessage):
    final_output: Union[Evaluation, ConversationalResponse]

change = 100

In [48]:
from langchain_core.prompts import ChatPromptTemplate

system_message_template = """
You are a python code quality evaluator.
1. Read the question at the end of this message.
2. Then you will receive a code snippet by user periodically. 
3. You have to evaluate the quality of code and give the output.
4. Restraint from giving the solution or suggestion or hints.

-------------------

Question: Write a program that takes input first name and last name from the user and passes it to the function greeting_full_name to print the full name of the user with a greeting.
"""

user_template = """
{code}
"""

change += 1

user_prompt = ChatPromptTemplate(["user",user_template])

In [49]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from langgraph.graph import START, MessagesState, StateGraph
import time

model = ChatOpenAI(temperature=0)

llm = model.with_structured_output(FinalResponse)

workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    print("RESPONSE", response)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

In [50]:
ACTIVE_THREAD_ID = f"test{change}"

config = {"configurable":{"thread_id":ACTIVE_THREAD_ID}}

def get_code(filename:str) -> str:
    with open(filename) as f:
        s = f.read()
        return s
    
def get_user_code_snippet() -> list[BaseMessage]:
    code_snippet = get_code("test.py")
    return user_prompt.invoke({"code": code_snippet}).to_messages()


def get_output(messages: list[BaseMessage]):
    output = app.invoke({"messages": messages}, config)
    print("OUTPUT", output)
    output["messages"][-1].pretty_print()

In [51]:
system_message = SystemMessage(system_message_template)

get_output(system_message)

ValidationError: 1 validation error for FinalResponse
final_output
  Field required [type=missing, input_value={'content': "def greeting..._name)", 'type': 'code'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/missing

In [23]:
while True:
    user_message = get_user_code_snippet()
    get_output(user_message)
    time.sleep(5)

Error parsing response: 1 validation error for Generation
text
  Input should be a valid string [type=string_type, input_value=FinalResponse(final_outpu...fied in the question.')), input_type=FinalResponse]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type
================================ System Message ================================

Error in parsing response


KeyboardInterrupt: 